In [1]:
!python --version


Python 3.9.13


In [ ]:
!pip install psycopg2-binary pyspark


In [1]:
import pandas as pd
import boto3
import sqlalchemy
import psycopg2
import pyspark

print("✅ All good! Libraries are installed and working.")


ModuleNotFoundError: No module named 'psycopg2'

In [ ]:
!pip install psycopg2-binary pyspark

In [1]:
import psycopg2

print("✅ All good! Libraries are installed and working.")

ModuleNotFoundError: No module named 'psycopg2'

In [1]:
!pip install psycopg2-binary

  Using cached psycopg2_binary-2.9.10-cp39-cp39-win_amd64.whl (1.2 MB)


In [ ]:
!pip install pyspark

In [1]:
import pandas as pd
import boto3
import sqlalchemy
import psycopg2
import pyspark

print("✅ All good! Libraries are installed and working.")

ModuleNotFoundError: No module named 'pyspark'

In [2]:
!pip install pyspark

In [3]:
import pandas as pd
import boto3
import sqlalchemy
import psycopg2
import pyspark

print("✅ All good! Libraries are installed and working.")

✅ All good! Libraries are installed and working.


In [4]:
import pandas as pd

url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet"
df = pd.read_parquet(url)
print(df.shape)


ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

In [5]:
!pip install pyarrow


     ---------------------------------------- 26.2/26.2 MB 5.6 MB/s eta 0:00:00


In [6]:
import pandas as pd

url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet"
df = pd.read_parquet(url)
print(df.shape)

(2964624, 19)


In [7]:
import pandas as pd

# Basic info
print("Shape:", df.shape)
print("\nColumns:", df.columns.tolist())

# Example transformations
df = df[df["fare_amount"] > 0]   # remove invalid fares
df = df[df["trip_distance"] > 0] # remove trips with zero distance

# Convert datetime columns
df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"])
df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"])

# Add derived column: trip duration in minutes
df["trip_duration_min"] = (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]).dt.total_seconds() / 60

# Preview cleaned data
print(df.head())
print("Shape after cleaning:", df.shape)


Shape: (2964624, 19)

Columns: ['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'Airport_fee']
   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         2  2024-01-01 00:57:55   2024-01-01 01:17:43              1.0   
1         1  2024-01-01 00:03:00   2024-01-01 00:09:36              1.0   
2         1  2024-01-01 00:17:06   2024-01-01 00:35:01              1.0   
3         1  2024-01-01 00:36:38   2024-01-01 00:44:56              1.0   
4         1  2024-01-01 00:46:51   2024-01-01 00:52:57              1.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           1.72         1.0                  N           186            79   
1           1.80         1.0     

In [8]:
df.to_csv("yellow_tripdata_clean.csv", index=False)
print("✅ Cleaned dataset saved as 'yellow_tripdata_clean.csv'")


✅ Cleaned dataset saved as 'yellow_tripdata_clean.csv'


In [9]:
# Jupyter cell
import sqlite3
import pandas as pd

conn = sqlite3.connect("etl_demo.db")
df = pd.read_csv("yellow_tripdata_clean.csv")     # or use existing df
df.to_sql("yellow_tripdata", conn, if_exists="replace", index=False)
print("✅ Loaded into etl_demo.db -> table yellow_tripdata")


C:\Users\Manideep kante\AppData\Local\Temp\ipykernel_14792\497722936.py:6: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("yellow_tripdata_clean.csv")     # or use existing df


✅ Loaded into etl_demo.db -> table yellow_tripdata


In [10]:
import time
import pandas as pd
import numpy as np

def time_query(sql, conn, runs=5):
    times = []
    for _ in range(runs):
        t0 = time.perf_counter()
        pd.read_sql(sql, conn)
        t1 = time.perf_counter()
        times.append(t1 - t0)
    return float(np.median(times)), float(np.mean(times))

# Example query: trips in the first week of Jan 2024
sql = """
SELECT COUNT(*) as trips
FROM yellow_tripdata
WHERE tpep_pickup_datetime BETWEEN '2024-01-01' AND '2024-01-07'
"""
baseline_median, baseline_mean = time_query(sql, conn, runs=3)
print("Baseline median (s):", baseline_median)
print("Baseline mean (s):", baseline_mean)


Baseline median (s): 0.9445329000000129
Baseline mean (s): 0.9389428666666694


In [11]:
cursor = conn.cursor()
cursor.execute("CREATE INDEX IF NOT EXISTS idx_pickup_date ON yellow_tripdata(tpep_pickup_datetime)")
conn.commit()
print("✅ Index created on pickup datetime")

indexed_median, indexed_mean = time_query(sql, conn, runs=3)
print("Indexed median (s):", indexed_median)
print("Indexed mean (s):", indexed_mean)

reduction_pct = (baseline_mean - indexed_mean) / baseline_mean * 100
print(f"Query time reduced by: {reduction_pct:.1f}%")


✅ Index created on pickup datetime
Indexed median (s): 0.061535100000128296
Indexed mean (s): 0.0565297333334153
Query time reduced by: 94.0%


In [12]:
explain = pd.read_sql("EXPLAIN QUERY PLAN " + sql, conn)
print(explain)


   id  parent  notused                                             detail
0   3       0        0  SEARCH yellow_tripdata USING COVERING INDEX id...


In [13]:
import boto3

s3 = boto3.client("s3", region_name="eu-north-1")
bucket = "etl-demo-mani"

s3.upload_file("yellow_tripdata_clean.csv", bucket, "processed/yellow_tripdata_clean.csv")
print("✅ File uploaded successfully to S3")


✅ File uploaded successfully to S3


In [14]:
response = s3.list_objects_v2(Bucket=bucket, Prefix="processed/")
for obj in response.get("Contents", []):
    print(obj["Key"])


processed/yellow_tripdata_clean.csv
